In [3]:
import intera_interface
import rospy
import json
import tf
from copy import deepcopy
from sawyer import transformations
from os import environ

In [4]:
environ['ROS_MASTER_URI'] = 'http://021607CP00116.local:11311'

In [5]:
environ['ROS_HOSTNAME'] = 'mingew.local'

In [6]:
rospy.init_node("sawyer_example")

In [7]:
effector = "right_hand"
world = "world"

In [8]:
tfl = tf.TransformListener(interpolate=True, cache_time=rospy.Duration(1000))
tfb = tf.TransformBroadcaster()

In [9]:
rs = intera_interface.RobotEnable(intera_interface.CHECK_VERSION)

In [10]:
rs.enable()

[INFO] [1555260165.914352]: Robot Enabled


In [11]:
limb = intera_interface.Limb('right')

In [12]:
hope_lower_left = {u'right_j0': 1.611796875,
                   u'right_j1': -2.75679296875,
                   u'right_j2': -3.0216201171875,
                   u'right_j3': 1.296830078125,
                   u'right_j4': 1.161048828125,
                   u'right_j5': -1.98382421875,
                   u'right_j6': 1.0612880859375}

In [13]:
limb.move_to_neutral()

In [14]:
limb.move_to_joint_positions(hope_lower_left)
# Move to extreme lower left in JOINT space

# Compute sticks in cartesian space

In [13]:
# Base is ["lower"]["left"][0] for HOPE
base_position = [-0.5, -0.7, 0.35]


In [14]:
# Starting from lower left (drawing direction)
# 
sticks = {"hope": [], "fear": []}
y_interval = 0.01     # Interval of sticks in y direction
z_interval = 0.02      # Interval of sticks in z direction
group_interval = 0.01  # Interval between each group in y direction
num_groups = 8         # Number of stick groups per line
num_sticks = 4         # Number of sticks per group
height_stick = 0.03    # Length in z direction
num_height = 8         # Number of sticks in z direction (in height)
num_points_cart = 20   # Number of cartesian points for the actual drawing motion

emotion = "fear"
z_stick = base_position[2]
col = 0
for h in range(num_height):
    y_stick = base_position[1]
    for g in range(num_groups):
        for i in range(num_sticks):
            # Add the end stick
            end_stick = [base_position[0], y_stick, z_stick + height_stick]
            tfb.sendTransform(end_stick, [0, 0, 0, 1], rospy.Time.now(), str(col), "world")
            motion = [end_stick]

            # Add the cartesian motion
            for p in range(num_points_cart):
                motion_point = deepcopy(end_stick)
                motion_point[2] = z_stick + height_stick - height_stick * (p+1) / num_points_cart
                motion.append(motion_point)
                #if i == 0 and g == 0:
                #    tfb.sendTransform(motion_point, [0, 0, 0, 1], rospy.Time.now(), str(col)+'-'+str(p), "world")
            
            sticks[emotion].append(motion)
            y_stick += y_interval
            col += 1
        y_stick += group_interval
    z_stick += z_interval + height_stick
    if len(sticks["fear"]) == num_groups*num_height*num_sticks/2:
        emotion = "hope"
        z_stick += 0.02
print(len(sticks["hope"]), "+", len(sticks["fear"]), "sticks generated")

(128, '+', 128, 'sticks generated')


In [15]:
sticks['fear'][0]

[[-0.5, -0.7, 0.38],
 [-0.5, -0.7, 0.3785],
 [-0.5, -0.7, 0.377],
 [-0.5, -0.7, 0.3755],
 [-0.5, -0.7, 0.374],
 [-0.5, -0.7, 0.3725],
 [-0.5, -0.7, 0.371],
 [-0.5, -0.7, 0.3695],
 [-0.5, -0.7, 0.368],
 [-0.5, -0.7, 0.3665],
 [-0.5, -0.7, 0.365],
 [-0.5, -0.7, 0.3635],
 [-0.5, -0.7, 0.362],
 [-0.5, -0.7, 0.3605],
 [-0.5, -0.7, 0.359],
 [-0.5, -0.7, 0.3575],
 [-0.5, -0.7, 0.356],
 [-0.5, -0.7, 0.3545],
 [-0.5, -0.7, 0.353],
 [-0.5, -0.7, 0.35150000000000003],
 [-0.5, -0.7, 0.35]]

# Cartesian sticks

In [16]:
hand_T_stick = [[0.14239612794977008, 0.01701314337065485, -0.07630844138995452],
 [0.5270677868272005,
  0.023880201241985057,
  -0.8461224120032248,
  0.07553904939543005]]

In [17]:
stick_fk = {"hope" : [], "fear": [], "joints": []}
seed = None
for emotion in ["hope", "fear"]:
    for stick_id, stick in enumerate(sticks[emotion]):
        motion = []
        for point_id, point in enumerate(stick):
            hand_T_world = transformations.multiply_transform([point, [0,0,0,1]], hand_T_stick)
            tfb.sendTransform(hand_T_world[0], hand_T_world[1], rospy.Time.now(), "eik", world)
            result = limb.ik_request(transformations.list_to_pose(hand_T_world), joint_seed=seed)
            if result == False:
                raise RuntimeError(emotion + ": can't compute stick start " + str(stick_id))
            else:
                motion.append(result.values())
                seed = result
        stick_fk[emotion].append(motion)
        stick_fk["joints"] = result.keys()
        #rospy.sleep(0.1)

In [30]:
with open("../config/motions.json", "w") as f:
    json.dump(stick_fk, f)

# Reload cartesian sticks

In [11]:
with open("../config/motions.json") as f:
    stick_fk = json.load(f)

In [12]:
from intera_motion_interface import (
    MotionTrajectory,
    MotionWaypoint,
    MotionWaypointOptions
)

In [13]:
traj  = MotionTrajectory()

In [24]:
traj.set_joint_names(stick_fk["joints"])
traj.

In [25]:
for point in stick_fk["hope"][0]:
    waypoint = MotionWaypoint(ma)
    waypoint.set_joint_angles(point)
    traj.append_waypoint(waypoint)

In [27]:
traj.send_trajectory(timeout=20)

result: True
errorId: ''
trajectory_analysis: 
  planned_duration: 4.09254180956
  measured_duration: 4.099870243
  min_angle_command: [0.6436005593905485, -1.8083957825243826, 1.04663495132743, 1.0935538557482565, -3.0225999873378693, -2.549719594979685, 1.5895995374984153]
  max_angle_command: [0.6805687632735085, 1.1754943508222875e-38, 1.0553534042121082, 1.109561937467399, 1.1754943508222875e-38, 1.1754943508222875e-38, 1.6007634731370282]
  peak_speed_command: [0.1516398975064467, 0.10858441551787587, 0.035762209929474084, 0.06566355988529682, 0.07535013896571861, 0.12243219121370731, 0.045793356369000254]
  peak_accel_command: [1.2253422532503047, 0.8774278706755578, 0.28898032520978995, 0.5306013506298057, 0.6088747788757376, 0.9893262889194343, 0.3700380664969747]
  peak_jerk_command: [10.649847802654358, 7.748108409003397, 4.830533143059915, 5.124832771628908, 5.646023849003435, 8.248532415716207, 3.5333986226465397]
  min_time_rate: 1.0
  max_time_rate: 1.0
  max_position_er

In [28]:
traj.clear_waypoints()

In [29]:
w = MotionWaypoint()

In [30]:
traj.set_joint_names(hope_lower_left.keys())
w.set_joint_angles(hope_lower_left.values())

In [31]:
traj.append_waypoint(w)

In [32]:
traj.send_trajectory(timeout=10)

result: True
errorId: ''
trajectory_analysis: 
  planned_duration: 0.92698697522
  measured_duration: 0.929326811
  min_angle_command: [0.6805718709535935, -1.9838242187594355, 1.046635579108908, 1.1095629499611237, -3.0216201132786344, -2.756792968581033, 1.5895996571901037]
  max_angle_command: [1.0612880859375, 1.1754943508222875e-38, 1.1610488281253692, 1.2968300781249995, 1.1754943508222875e-38, 1.1754943508222875e-38, 1.6117968738349906]
  peak_speed_command: [0.7084717308368675, 0.37842880559743924, 0.2468132366852051, 0.40397512051011786, 0.0375130937997645, 0.44669918035683526, 0.04788414741695481]
  peak_accel_command: [2.525521240364423, 1.2517726003458554, 0.9414482266034151, 1.6576309408666876, 0.16054496039042834, 1.91174054050629, 0.20493000634441685]
  peak_jerk_command: [23.152732604838953, 7.768764871376634, 4.677076241363913, 7.543246353955295, 0.698587915175077, 8.318659366822375, 0.8917229512580622]
  min_time_rate: 1.0
  max_time_rate: 1.0
  max_position_error: [0

# Overwrite other poses

In [ ]:
# Pause pose
poses = {"pause": limb.joint_angles()}

with open("../config/poses.json", "w") as f:
    json.dump(poses, f, indent=4)

In [17]:
with open("../config/poses.json") as f:
    poses = json.load(f)

In [ ]:
limb.move_to_joint_positions(poses["pause"])

In [18]:
poses

{u'pause': {u'right_j0': 2.462169921875,
  u'right_j1': -3.327517578125,
  u'right_j2': -2.2095576171875,
  u'right_j3': -1.52369140625,
  u'right_j4': 0.1305107421875,
  u'right_j5': -1.00734375,
  u'right_j6': -0.2455576171875}}

# Display sticks

In [ ]:
with open("../config/sticks_target_ik.json") as f:
    sticks = json.load(f)

In [ ]:
from sensor_msgs.msg import JointState
p = rospy.Publisher("display/joint_states", JointState, queue_size=1)

In [ ]:
type = 'fear'
side = 'end'
for i, pose in enumerate(sticks[type]):
    js = JointState()
    js.header.stamp = rospy.Time.now()
    js.name = pose[side].keys()
    js.position = pose[side].values()
    p.publish(js)
    input = raw_input('Press to display {} pose {} of [q to quit] '.format(side, i, type))
    if input == 'q': break

In [ ]:
sticks